<a href="https://colab.research.google.com/github/denizMoC/MetabolismofCities/blob/main/S_Sankey_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MIT License
#

Copyright (c) [2023] [Metabolism of Cities]
#

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
#


THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
#



Your main code starts after Libraries code section, you have to authenticate your google account to use this tool.
#


Data code section starts with importing your google docs sheet to visualize your Sankey diagram. Google docs sheet should have "FROM", "TO", "value", "flow_label", "flow_color", "node_color" columns for every indicator. You should replace LINK with the link of your google sheets from this data code. (worksheet = gc.open_by_url('LINK').sheet1)
#


To export the HTML file of your Sankey diagram, first run the code and go to files on the left pane. You will see "Sankey - Template.html" after running the code which is saved in the current working directory of your Google Colab notebook. Hover on file and click on the three dots button in the top right corner. Select "Download" from the drop-down menu and save your Sankey diagram HTML file to your local machine.  

Libraries 

In [ ]:
import plotly.graph_objects as go

from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()


gc = gspread.authorize(creds)

## Data

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SYxsQ7OJ5yEa16TQI7oKsZMYwQ28JDGntU7BqdmltTk/edit?usp=sharing').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows[1:])

#Adding the headers
df.set_axis(["FROM", "TO", "value", "label", "flow_color","node_color"],axis=1,inplace=True)
df

[['From', 'To', 'Value', 'Flow Label', 'flow_color', 'node_color'], ['Imports', 'Exports', '2.41', 'Biomass', 'rgba(56, 118, 30, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Imports', 'Exports', '4.26', 'Metal ores and metals', 'rgba(241, 194, 51, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Imports', 'Exports', '107.71', 'Non-metallic minerals', 'rgba(224, 102, 102, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Imports', 'Exports', '85.45', 'Fossil energy carriers', 'rgba(142, 124, 195, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Domestic Extraction', 'Exports', '6.38', 'Biomass', 'rgba(56, 118, 30, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Domestic Extraction', 'Exports', '0.00', 'Metal ores and metals', 'rgba(241, 194, 51, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Domestic Extraction', 'Exports', '1.74', 'Non-metallic minerals', 'rgba(224, 102, 102, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Domestic Extraction', 'Exports', '0.00', 'Fossil energy carriers', 'rgba(142, 124, 195, 0.5)', 'rgba(239, 239, 239, 0.5)'], ['Imports

,FROM,TO,value,label,flow_color,node_color
0,Imports,Exports,2.41,Biomass,"rgba(56, 118, 30, 0.5)","rgba(239, 239, 239, 0.5)"
1,Imports,Exports,4.26,Metal ores and metals,"rgba(241, 194, 51, 0.5)","rgba(239, 239, 239, 0.5)"
2,Imports,Exports,107.71,Non-metallic minerals,"rgba(224, 102, 102, 0.5)","rgba(239, 239, 239, 0.5)"
3,Imports,Exports,85.45,Fossil energy carriers,"rgba(142, 124, 195, 0.5)","rgba(239, 239, 239, 0.5)"
4,Domestic Extraction,Exports,6.38,Biomass,"rgba(56, 118, 30, 0.5)","rgba(239, 239, 239, 0.5)"
...,...,...,...,...,...,...
67,EoL Waste,Incineration with energy recovery,1.36,Fossil energy carriers,"rgba(142, 124, 195, 0.5)","rgba(239, 239, 239, 0.5)"
68,EoL Waste,Backfilling,0.00,Biomass,"rgba(56, 118, 30, 0.5)","rgba(239, 239, 239, 0.5)"
69,EoL Waste,Backfilling,0.00,Metal ores and metals,"rgba(241, 194, 51, 0.5)","rgba(239, 239, 239, 0.5)"
70,EoL Waste,Backfilling,0.00,Non-metallic minerals,"rgba(224, 102, 102, 0.5)","rgba(239, 239, 239, 0.5)"


In [ ]:
# We create the 3 list we need
source = df['FROM'].tolist()
target = df['TO'].tolist()
flow_color = df['flow_color'].tolist()
node_color = df['node_color'].tolist()
value = df['value'].tolist()
node = pd.unique(source + target).tolist()

# Function to create a list of unique nodes and assign them 0 based number
def assign_number(node, source, target):
  """
  It takes in a list of nodes, a list of source nodes, and a list of target nodes, and returns a list
  of source nodes and a list of target nodes, where each node is replaced by its index in the list of
  nodes
  
  :param node: the list of nodes in the graph
  :param source: the source node
  :param target: the target node
  :return: The source and target lists are being returned.
  """
  for i in range(len(node)):
      for j in range(len(source)):
          if node[i] == source[j]:
              source[j] = i
      for k in range(len(target)):
          if node[i] == target[k]:
              target[k] = i
  return source, target

source, target = assign_number(node, source, target) # assign number to source and target

print(node)
print(source)
print(target)
print(value)

['Imports', 'Domestic Extraction', 'Processed materials', 'Energetic use', 'EoL Waste', 'Material use', 'In-use stocks', 'Recovery', 'Exports', 'DPO emissions', 'Landfill', 'Incineration', 'Incineration with energy recovery', 'Backfilling']
[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 5, 5, 5, 5, 9, 9, 9, 9, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 4, 4, 4, 4, 2, 2, 2, 2, 8, 8, 8, 8, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13]
['2.41', '4.26', '107.71', '85.45', '6.38', '0.00', '1.74', '0.00', '16.70', '6.96', '182.72', '85.45', '44.15', '0.00', '2.96', '0.00', '58.17', '0.00', '0.00', '85.48', '16.51', '7.89', '187.49', '3.56', '20.24', '0.00', '0.00', '85.07', '15.84', '1.50', '2.87', '4.64', '37.93', '0.00', '0.00', '0.42', '8.

In [ ]:
# Function to plot a sankey diagram

fig = go.Figure(data=[go.Sankey(
    node = dict(
        # You can set a whole lot of stylistic options, see https://plotly.com/python/reference/sankey/
        thickness = 10,
        line = dict(width = 0),
        color = node_color,
        # This is the list of nodes: the TO and FROM in your spreadsheet
        # Each node must be included once and only once
        label = node,
    ),
    # These are the connections between the nodes, the flows between them
    # They're made up of three parts: the source, the target, and the value
    # The numbers correspond to the labels (0-based)
    # So, the first item of each array is taken to form the first connection
    # The it continues with the second, third, and so forth until the end
    link = dict(
      source= source,
      target = target,
      value = value,
      color = flow_color
      # You can optionally add labels that will show up when hovering over the connection
      # For connections without a label, simply put quotation marks without any text: ""
      # labels = ["label for first connection", "", "", ...]
  ))])
# This updates some layout stuff so that it matches MOC styles
# See https://plotly.com/python/reference/sankey/ for options
fig.update_layout(
    hovermode = 'x',
    font=dict(size = 14, color = 'black', family = 'Lato'),
    #plot_bgcolor='rgba(255,255,255,0)',
    #paper_bgcolor='rgba(255,255,255,0)',
)

fig.update_traces(
    node_hoverlabel_bgcolor='rgba(255,255,255,0)',    # background color of the hover label for nodes
    link_hoverlabel_bgcolor='rgba(255,255,255,0)'     # background color of the hover label for flows

)

sankey = fig.to_html(full_html=True)
context = {
    "sankey": sankey,
}

fig.update_layout(title_text="", font_size=10)

#To change the arrangement
fig.update_traces(arrangement="freeform", 
                  node = {"label": node,
                              "x": [0,    0, 0.25,0.5, 0.5, 0.65, 0.8,   1,   1,   1],
                              "y": [1, 0.01,  0.7,0.7, 0.9,  0.9, 0.6, 0.1, 0.8, 0.7],
                          'pad':100},  # 10 Pixels 
                  
node_pad=15, 
selector=dict(type='sankey'))

# Export the HTML
fig.write_html("Sankey - Template.html")


#To display
fig.show()
